<a href="https://colab.research.google.com/github/misharigot/kobe/blob/master/src/model/nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains the neural network to predict kobe's shots.

## To Do from Trello
- [x] Implementeren van cross validation.
- [ ] Connecten van nieuwe cross validation module met de nn model module.
- [ ] Bouwen van verschillende netwerken (vorm, aantal nodes etc.)
- [ ] Kijken welke loss function we moeten gebruiken, cross entropy vs log loss. Log loss sowieso proberen om te vergelijken met competition entries.
- [ ] Implementeren van model export functie.

In [2]:
# When using this notebook in Google Colab, clone the repo in the file system in
# order to use the python modules from the repo.
!git  clone https://github.com/misharigot/kobe.git

Cloning into 'kobe'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 178 (delta 74), reused 126 (delta 34), pack-reused 0
Receiving objects: 100% (178/178), 1.36 MiB | 19.90 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [34]:
import sys; sys.path.insert(0, '..')  # Needed to make the import below work

# Use the line below in Colab
# from kobe.src.multiple_train_test_splits import MultipleTrainTestSplits

# Use the line below in a local env
from multiple_train_test_splits import MultipleTrainTestSplits
from preprocessor import Preprocessor

import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing

In [35]:
def get_x(data: pd.DataFrame) -> pd.DataFrame:
    """Returns the features.
    """
    X = data.drop(columns=['shot_made_flag'])
    return X

def get_y(data: pd.DataFrame) -> pd.Series:
    """Returns the target.
    """
    Y = data['shot_made_flag'].copy()
    return Y


In [36]:
def create_model_1(input_dim: int):
    """Simple one hidden layer network.
    """
    model = Sequential()

    model.add(Dense(units=32, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.add(Dropout(0.5))
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model


def create_model_2(input_dim: int):
    """2 hidden layers network.
    """
    model = Sequential()

    model.add(Dense(units=64, activation='relu', input_dim=input_dim))
    model.add(Dropout(0.5))
    model.add(Dense(units=32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model


def create_model_3(input_dim: int):
    """1 hidden layer network with a lot of neurons.
    """
    model = Sequential()

    model.add(Dense(units=int(input_dim/2), activation='relu', input_dim=input_dim))
    model.add(Dropout(0.5))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model


def get_models_dict():
    models = {}
    models['model_1'] = create_model_1
    # models['model_2'] = create_model_2
#     models['model_3'] = create_model_3
    return models


In [37]:
# import datetime as dt
# first_recorded_game = str(dt.datetime.strptime(
#             min(pp.raw_data['game_date']), '%Y-%m-%d').strftime('%Y-%m-%d'))
# print(first_recorded_game)

In [38]:
# Use in Colab
# csv_path = 'kobe/data/data.csv'

# Use in local env
csv_path = '../../data/data.csv'

mtts = MultipleTrainTestSplits(csv_path=csv_path)
pp = Preprocessor(path_to_raw_data=csv_path)

test_set = mtts.test_set

loss_and_metrics = {}
models = get_models_dict()

# Loop over the models
for model_name, model_func in models.items():
    # checkpoint_path = "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

    # Loop over the train/validation splits/folds
    n_fold = 0
    for train_set, validation_set in mtts.train_validation_split(as_dataframe=True):
        n_fold = n_fold + 1
        print(f'Training model: {model_name}, Fold: {n_fold}')
        checkpoint_path = f"{model_name}_fold_{n_fold}_weights-improvement" + "-{epoch:02d}-{val_acc:.2f}.hdf5"

        # Preprocess the training set
        preprocessed_train_set = pp.preprocess(train_set)
        # Split the features from the target
        x_train = get_x(preprocessed_train_set)
        y_train = get_y(preprocessed_train_set)

        # Preprocess the validation set
        preprocessed_validation_set= pp.preprocess(validation_set)
        # Split the features from the target
        x_validation = get_x(preprocessed_validation_set)
        y_validation = get_y(preprocessed_validation_set)

        input_dim = x_train.shape[1]  # number of columns (dimensions for the input layer of the model)
        
        # model = create_model(input_dim=input_dim)
        model = model_func(input_dim)

#         # Create model checkpoint to be able to resume at a checkpoint when training crashes.
#         checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
#         callbacks_list = [checkpoint]

        # Fit the model
#         model.fit(x_train, y_train, epochs=2, batch_size=10, 
#                   validation_data=(x_validation, y_validation),
#                   callbacks=callbacks_list, verbose=0)
        model.fit(x_train, y_train, epochs=2, batch_size=10)
    
        loss_and_metrics[model_name] = {
            n_fold: model.evaluate(x_validation, y_validation, batch_size=128)
        }
    

Training model: model_1, Fold: 1
object
Epoch 1/2
3180/5141 [=================>............] - ETA: 1s - loss: 4.9074 - accuracy: 0.5009

KeyboardInterrupt: 

In [20]:
loss_and_metrics

{'model_1': {3: [0.8469062415884032, 0.542907178401947]}}

In [17]:
def print_average_metrics(loss_and_metrics):
    # Get average accuracy
    
    for k, v in loss_and_metrics.items():
        
#     accuracies = []
#     for row in loss_and_metrics:
#         accuracies.append(row[1])
#     avg_accuracy = sum(accuracies) / len(accuracies)

#     print('Average accuracy:', round(avg_accuracy, 4))


print_average_metrics(loss_and_metrics)

model_3_fold_1 [0.8702678171130186, 0.4384121298789978]
model_3_fold_2 [0.6855128304135778, 0.6275539994239807]
model_3_fold_3 [0.6699073620982893, 0.6191866397857666]


In [2]:
!ls -lsa

total 248
  0 drwxr-xr-x  7 Misha  staff    224 Mar 28 21:21 .
  0 drwxr-xr-x  9 Misha  staff    288 Mar 28 17:35 ..
  0 drwxr-xr-x  6 Misha  staff    192 Mar 26 23:41 .ipynb_checkpoints
 32 -rw-r--r--  1 Misha  staff  15084 Mar 28 14:22 decision_tree.ipynb
144 -rw-r--r--  1 Misha  staff  70555 Mar 26 23:41 knn_classifier.ipynb
 24 -rw-r--r--  1 Misha  staff   9549 Mar 28 20:08 knn_v2.ipynb
 48 -rw-r--r--  1 Misha  staff  23091 Mar 28 21:21 nn.ipynb


In [ ]:
classes = model.predict(x_validation, batch_size=128)
classes

array([[0.26074135],
       [0.2928418 ],
       [0.2698071 ],
       ...,
       [0.29890507],
       [0.9255194 ],
       [0.4461364 ]], dtype=float32)